In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk

In [11]:
df = pd.read_csv('articles.csv')
df['publishedAt'] = pd.to_datetime(df['publishedAt'])
df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'MarketWatch'}",Philip van Doorn,Elon Musk has egg on his face,"Even Wedbush analyst Daniel Ives, normally a r...",https://www.marketwatch.com/story/elon-musk-ha...,https://images.mktw.net/im-18501351/social,2024-01-26 16:03:00+00:00,Shares of Tesla Inc. were down 26.5% for 2024 ...
1,"{'id': None, 'name': 'MarketWatch'}",Steve Goldstein,Tesla cuts prices in Germany after reductions ...,Tesla cut the prices of its Model Y in Germany...,https://www.marketwatch.com/story/tesla-cuts-p...,https://images.mktw.net/im-795874/social,2024-01-17 10:22:00+00:00,Tesla cut the prices of its Model Y in Germany...
2,"{'id': None, 'name': 'MarketWatch'}",Claudia Assis,Tesla warns Wall Street it could sell fewer ve...,Tesla Inc. warned investors late Wednesday tha...,https://www.marketwatch.com/story/tesla-warns-...,https://images.mktw.net/im-475346/social,2024-01-24 21:16:00+00:00,Tesla Inc. warned investors late Wednesday tha...
3,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Nio’s stock tanks toward a record-tying loss s...,Nio's stock on Wednesday dove to its lowest pr...,https://www.marketwatch.com/story/nios-stock-t...,https://images.mktw.net/im-19267906/social,2024-01-17 16:44:00+00:00,Shares of Nio Inc. on Wednesday dove to its lo...
4,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Tesla’s stock hasn’t fallen enough – this anal...,Although Tesla’s stock already took a sound be...,https://www.marketwatch.com/story/teslas-stock...,https://images.mktw.net/im-72300848/social,2024-01-26 13:37:00+00:00,Although Tesla Inc.s stock already took a soun...
...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Gaining 20% In 2023 Will Delta Continue To Out...,"In this note, we discuss why we believe that D...",https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-10 13:00:00+00:00,"LOS ANGELES, CA - DECEMBER 29: Southwest Airli..."
96,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Up Over 2x In 2023 Is AMD A Better Pick Over U...,Although these companies are from different se...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-23 14:00:00+00:00,"ROUND ROCK, TEXAS - APRIL 21: A Union Pacific ..."
97,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...","Up 130% In The Last 12 Months, Will AI Power A...",We project that earnings will come in at about...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-22 12:00:00+00:00,"DANA POINT, CALIFORNIA - SEPTEMBER 26: Dr. Lis..."
98,"{'id': None, 'name': 'CleanTechnica'}",Carolyn Fortuna,Tesla’s Recent Q4 & FY 2023 Update Makes Stock...,"If you’re a small investor in Tesla like I am,...",https://cleantechnica.com/2024/01/29/teslas-re...,https://cleantechnica.com/wp-content/uploads/2...,2024-01-29 12:18:58+00:00,Sign up for daily news updates from CleanTechn...


In [12]:
# Lọc DataFrame để chỉ giữ lại các dòng có 'date' sau hoặc bằng 1/1/2024
df = df[df['publishedAt'] >= '2024-01-01']
df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'MarketWatch'}",Philip van Doorn,Elon Musk has egg on his face,"Even Wedbush analyst Daniel Ives, normally a r...",https://www.marketwatch.com/story/elon-musk-ha...,https://images.mktw.net/im-18501351/social,2024-01-26 16:03:00+00:00,Shares of Tesla Inc. were down 26.5% for 2024 ...
1,"{'id': None, 'name': 'MarketWatch'}",Steve Goldstein,Tesla cuts prices in Germany after reductions ...,Tesla cut the prices of its Model Y in Germany...,https://www.marketwatch.com/story/tesla-cuts-p...,https://images.mktw.net/im-795874/social,2024-01-17 10:22:00+00:00,Tesla cut the prices of its Model Y in Germany...
2,"{'id': None, 'name': 'MarketWatch'}",Claudia Assis,Tesla warns Wall Street it could sell fewer ve...,Tesla Inc. warned investors late Wednesday tha...,https://www.marketwatch.com/story/tesla-warns-...,https://images.mktw.net/im-475346/social,2024-01-24 21:16:00+00:00,Tesla Inc. warned investors late Wednesday tha...
3,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Nio’s stock tanks toward a record-tying loss s...,Nio's stock on Wednesday dove to its lowest pr...,https://www.marketwatch.com/story/nios-stock-t...,https://images.mktw.net/im-19267906/social,2024-01-17 16:44:00+00:00,Shares of Nio Inc. on Wednesday dove to its lo...
4,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Tesla’s stock hasn’t fallen enough – this anal...,Although Tesla’s stock already took a sound be...,https://www.marketwatch.com/story/teslas-stock...,https://images.mktw.net/im-72300848/social,2024-01-26 13:37:00+00:00,Although Tesla Inc.s stock already took a soun...
...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Gaining 20% In 2023 Will Delta Continue To Out...,"In this note, we discuss why we believe that D...",https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-10 13:00:00+00:00,"LOS ANGELES, CA - DECEMBER 29: Southwest Airli..."
96,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Up Over 2x In 2023 Is AMD A Better Pick Over U...,Although these companies are from different se...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-23 14:00:00+00:00,"ROUND ROCK, TEXAS - APRIL 21: A Union Pacific ..."
97,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...","Up 130% In The Last 12 Months, Will AI Power A...",We project that earnings will come in at about...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-22 12:00:00+00:00,"DANA POINT, CALIFORNIA - SEPTEMBER 26: Dr. Lis..."
98,"{'id': None, 'name': 'CleanTechnica'}",Carolyn Fortuna,Tesla’s Recent Q4 & FY 2023 Update Makes Stock...,"If you’re a small investor in Tesla like I am,...",https://cleantechnica.com/2024/01/29/teslas-re...,https://cleantechnica.com/wp-content/uploads/2...,2024-01-29 12:18:58+00:00,Sign up for daily news updates from CleanTechn...


In [13]:
df = df.drop_duplicates(subset='title')
df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'MarketWatch'}",Philip van Doorn,Elon Musk has egg on his face,"Even Wedbush analyst Daniel Ives, normally a r...",https://www.marketwatch.com/story/elon-musk-ha...,https://images.mktw.net/im-18501351/social,2024-01-26 16:03:00+00:00,Shares of Tesla Inc. were down 26.5% for 2024 ...
1,"{'id': None, 'name': 'MarketWatch'}",Steve Goldstein,Tesla cuts prices in Germany after reductions ...,Tesla cut the prices of its Model Y in Germany...,https://www.marketwatch.com/story/tesla-cuts-p...,https://images.mktw.net/im-795874/social,2024-01-17 10:22:00+00:00,Tesla cut the prices of its Model Y in Germany...
2,"{'id': None, 'name': 'MarketWatch'}",Claudia Assis,Tesla warns Wall Street it could sell fewer ve...,Tesla Inc. warned investors late Wednesday tha...,https://www.marketwatch.com/story/tesla-warns-...,https://images.mktw.net/im-475346/social,2024-01-24 21:16:00+00:00,Tesla Inc. warned investors late Wednesday tha...
3,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Nio’s stock tanks toward a record-tying loss s...,Nio's stock on Wednesday dove to its lowest pr...,https://www.marketwatch.com/story/nios-stock-t...,https://images.mktw.net/im-19267906/social,2024-01-17 16:44:00+00:00,Shares of Nio Inc. on Wednesday dove to its lo...
4,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Tesla’s stock hasn’t fallen enough – this anal...,Although Tesla’s stock already took a sound be...,https://www.marketwatch.com/story/teslas-stock...,https://images.mktw.net/im-72300848/social,2024-01-26 13:37:00+00:00,Although Tesla Inc.s stock already took a soun...
...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Gaining 20% In 2023 Will Delta Continue To Out...,"In this note, we discuss why we believe that D...",https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-10 13:00:00+00:00,"LOS ANGELES, CA - DECEMBER 29: Southwest Airli..."
96,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Up Over 2x In 2023 Is AMD A Better Pick Over U...,Although these companies are from different se...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-23 14:00:00+00:00,"ROUND ROCK, TEXAS - APRIL 21: A Union Pacific ..."
97,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...","Up 130% In The Last 12 Months, Will AI Power A...",We project that earnings will come in at about...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-22 12:00:00+00:00,"DANA POINT, CALIFORNIA - SEPTEMBER 26: Dr. Lis..."
98,"{'id': None, 'name': 'CleanTechnica'}",Carolyn Fortuna,Tesla’s Recent Q4 & FY 2023 Update Makes Stock...,"If you’re a small investor in Tesla like I am,...",https://cleantechnica.com/2024/01/29/teslas-re...,https://cleantechnica.com/wp-content/uploads/2...,2024-01-29 12:18:58+00:00,Sign up for daily news updates from CleanTechn...


In [14]:
# Define a function to clean the text
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

# Cleaning the text in the review column
df['cleaned text'] = df['content'].apply(clean)
df

,source,author,title,description,url,urlToImage,publishedAt,content,cleaned text
0,"{'id': None, 'name': 'MarketWatch'}",Philip van Doorn,Elon Musk has egg on his face,"Even Wedbush analyst Daniel Ives, normally a r...",https://www.marketwatch.com/story/elon-musk-ha...,https://images.mktw.net/im-18501351/social,2024-01-26 16:03:00+00:00,Shares of Tesla Inc. were down 26.5% for 2024 ...,Shares of Tesla Inc were down for through Thur...
1,"{'id': None, 'name': 'MarketWatch'}",Steve Goldstein,Tesla cuts prices in Germany after reductions ...,Tesla cut the prices of its Model Y in Germany...,https://www.marketwatch.com/story/tesla-cuts-p...,https://images.mktw.net/im-795874/social,2024-01-17 10:22:00+00:00,Tesla cut the prices of its Model Y in Germany...,Tesla cut the prices of its Model Y in Germany...
2,"{'id': None, 'name': 'MarketWatch'}",Claudia Assis,Tesla warns Wall Street it could sell fewer ve...,Tesla Inc. warned investors late Wednesday tha...,https://www.marketwatch.com/story/tesla-warns-...,https://images.mktw.net/im-475346/social,2024-01-24 21:16:00+00:00,Tesla Inc. warned investors late Wednesday tha...,Tesla Inc warned investors late Wednesday that...
3,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Nio’s stock tanks toward a record-tying loss s...,Nio's stock on Wednesday dove to its lowest pr...,https://www.marketwatch.com/story/nios-stock-t...,https://images.mktw.net/im-19267906/social,2024-01-17 16:44:00+00:00,Shares of Nio Inc. on Wednesday dove to its lo...,Shares of Nio Inc on Wednesday dove to its low...
4,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Tesla’s stock hasn’t fallen enough – this anal...,Although Tesla’s stock already took a sound be...,https://www.marketwatch.com/story/teslas-stock...,https://images.mktw.net/im-72300848/social,2024-01-26 13:37:00+00:00,Although Tesla Inc.s stock already took a soun...,Although Tesla Inc s stock already took a soun...
...,...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Gaining 20% In 2023 Will Delta Continue To Out...,"In this note, we discuss why we believe that D...",https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-10 13:00:00+00:00,"LOS ANGELES, CA - DECEMBER 29: Southwest Airli...",LOS ANGELES CA DECEMBER Southwest Airlines Boe...
96,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Up Over 2x In 2023 Is AMD A Better Pick Over U...,Although these companies are from different se...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-23 14:00:00+00:00,"ROUND ROCK, TEXAS - APRIL 21: A Union Pacific ...",ROUND ROCK TEXAS APRIL A Union Pacific freight...
97,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...","Up 130% In The Last 12 Months, Will AI Power A...",We project that earnings will come in at about...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-22 12:00:00+00:00,"DANA POINT, CALIFORNIA - SEPTEMBER 26: Dr. Lis...",DANA POINT CALIFORNIA SEPTEMBER Dr Lisa Su Cha...
98,"{'id': None, 'name': 'CleanTechnica'}",Carolyn Fortuna,Tesla’s Recent Q4 & FY 2023 Update Makes Stock...,"If you’re a small investor in Tesla like I am,...",https://cleantechnica.com/2024/01/29/teslas-re...,https://cleantechnica.com/wp-content/uploads/2...,2024-01-29 12:18:58+00:00,Sign up for daily news updates from CleanTechn...,Sign up for daily news updates from CleanTechn...


In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
# Load the English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from a text
def remove_stopwords(text):
    text = str(text)
    word_tokens = word_tokenize(text)  # Tokenize the text into words
    filtered_sentence = [word for word in word_tokens if word.casefold()  not in stop_words]  # Remove stopwords
    return ' '.join(filtered_sentence)  # Join the words back into a sentence

In [16]:
# Apply the function to the 'comment_text' column
df['no_sw_text'] = df['cleaned text'].apply(remove_stopwords)
df

,source,author,title,description,url,urlToImage,publishedAt,content,cleaned text,no_sw_text
0,"{'id': None, 'name': 'MarketWatch'}",Philip van Doorn,Elon Musk has egg on his face,"Even Wedbush analyst Daniel Ives, normally a r...",https://www.marketwatch.com/story/elon-musk-ha...,https://images.mktw.net/im-18501351/social,2024-01-26 16:03:00+00:00,Shares of Tesla Inc. were down 26.5% for 2024 ...,Shares of Tesla Inc were down for through Thur...,Shares Tesla Inc Thursday one thing investors ...
1,"{'id': None, 'name': 'MarketWatch'}",Steve Goldstein,Tesla cuts prices in Germany after reductions ...,Tesla cut the prices of its Model Y in Germany...,https://www.marketwatch.com/story/tesla-cuts-p...,https://images.mktw.net/im-795874/social,2024-01-17 10:22:00+00:00,Tesla cut the prices of its Model Y in Germany...,Tesla cut the prices of its Model Y in Germany...,Tesla cut prices Model Germany week cutting pr...
2,"{'id': None, 'name': 'MarketWatch'}",Claudia Assis,Tesla warns Wall Street it could sell fewer ve...,Tesla Inc. warned investors late Wednesday tha...,https://www.marketwatch.com/story/tesla-warns-...,https://images.mktw.net/im-475346/social,2024-01-24 21:16:00+00:00,Tesla Inc. warned investors late Wednesday tha...,Tesla Inc warned investors late Wednesday that...,Tesla Inc warned investors late Wednesday may ...
3,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Nio’s stock tanks toward a record-tying loss s...,Nio's stock on Wednesday dove to its lowest pr...,https://www.marketwatch.com/story/nios-stock-t...,https://images.mktw.net/im-19267906/social,2024-01-17 16:44:00+00:00,Shares of Nio Inc. on Wednesday dove to its lo...,Shares of Nio Inc on Wednesday dove to its low...,Shares Nio Inc Wednesday dove lowest prices se...
4,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Tesla’s stock hasn’t fallen enough – this anal...,Although Tesla’s stock already took a sound be...,https://www.marketwatch.com/story/teslas-stock...,https://images.mktw.net/im-72300848/social,2024-01-26 13:37:00+00:00,Although Tesla Inc.s stock already took a soun...,Although Tesla Inc s stock already took a soun...,Although Tesla Inc stock already took sound be...
...,...,...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Gaining 20% In 2023 Will Delta Continue To Out...,"In this note, we discuss why we believe that D...",https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-10 13:00:00+00:00,"LOS ANGELES, CA - DECEMBER 29: Southwest Airli...",LOS ANGELES CA DECEMBER Southwest Airlines Boe...,LOS ANGELES CA DECEMBER Southwest Airlines Boe...
96,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Up Over 2x In 2023 Is AMD A Better Pick Over U...,Although these companies are from different se...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-23 14:00:00+00:00,"ROUND ROCK, TEXAS - APRIL 21: A Union Pacific ...",ROUND ROCK TEXAS APRIL A Union Pacific freight...,ROUND ROCK TEXAS APRIL Union Pacific freight t...
97,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...","Up 130% In The Last 12 Months, Will AI Power A...",We project that earnings will come in at about...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-22 12:00:00+00:00,"DANA POINT, CALIFORNIA - SEPTEMBER 26: Dr. Lis...",DANA POINT CALIFORNIA SEPTEMBER Dr Lisa Su Cha...,DANA POINT CALIFORNIA SEPTEMBER Dr Lisa Su Cha...
98,"{'id': None, 'name': 'CleanTechnica'}",Carolyn Fortuna,Tesla’s Recent Q4 & FY 2023 Update Makes Stock...,"If you’re a small investor in Tesla like I am,...",https://cleantechnica.com/2024/01/29/teslas-re...,https://cleantechnica.com/wp-content/uploads/2...,2024-01-29 12:18:58+00:00,Sign up for daily news updates from CleanTechn...,Sign up for daily news update

In [17]:
df = df[~(df.apply(lambda row: (row == 'deleted').all(), axis=1))]
df

,source,author,title,description,url,urlToImage,publishedAt,content,cleaned text,no_sw_text
0,"{'id': None, 'name': 'MarketWatch'}",Philip van Doorn,Elon Musk has egg on his face,"Even Wedbush analyst Daniel Ives, normally a r...",https://www.marketwatch.com/story/elon-musk-ha...,https://images.mktw.net/im-18501351/social,2024-01-26 16:03:00+00:00,Shares of Tesla Inc. were down 26.5% for 2024 ...,Shares of Tesla Inc were down for through Thur...,Shares Tesla Inc Thursday one thing investors ...
1,"{'id': None, 'name': 'MarketWatch'}",Steve Goldstein,Tesla cuts prices in Germany after reductions ...,Tesla cut the prices of its Model Y in Germany...,https://www.marketwatch.com/story/tesla-cuts-p...,https://images.mktw.net/im-795874/social,2024-01-17 10:22:00+00:00,Tesla cut the prices of its Model Y in Germany...,Tesla cut the prices of its Model Y in Germany...,Tesla cut prices Model Germany week cutting pr...
2,"{'id': None, 'name': 'MarketWatch'}",Claudia Assis,Tesla warns Wall Street it could sell fewer ve...,Tesla Inc. warned investors late Wednesday tha...,https://www.marketwatch.com/story/tesla-warns-...,https://images.mktw.net/im-475346/social,2024-01-24 21:16:00+00:00,Tesla Inc. warned investors late Wednesday tha...,Tesla Inc warned investors late Wednesday that...,Tesla Inc warned investors late Wednesday may ...
3,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Nio’s stock tanks toward a record-tying loss s...,Nio's stock on Wednesday dove to its lowest pr...,https://www.marketwatch.com/story/nios-stock-t...,https://images.mktw.net/im-19267906/social,2024-01-17 16:44:00+00:00,Shares of Nio Inc. on Wednesday dove to its lo...,Shares of Nio Inc on Wednesday dove to its low...,Shares Nio Inc Wednesday dove lowest prices se...
4,"{'id': None, 'name': 'MarketWatch'}",Tomi Kilgore,Tesla’s stock hasn’t fallen enough – this anal...,Although Tesla’s stock already took a sound be...,https://www.marketwatch.com/story/teslas-stock...,https://images.mktw.net/im-72300848/social,2024-01-26 13:37:00+00:00,Although Tesla Inc.s stock already took a soun...,Although Tesla Inc s stock already took a soun...,Although Tesla Inc stock already took sound be...
...,...,...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Gaining 20% In 2023 Will Delta Continue To Out...,"In this note, we discuss why we believe that D...",https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-10 13:00:00+00:00,"LOS ANGELES, CA - DECEMBER 29: Southwest Airli...",LOS ANGELES CA DECEMBER Southwest Airlines Boe...,LOS ANGELES CA DECEMBER Southwest Airlines Boe...
96,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...",Up Over 2x In 2023 Is AMD A Better Pick Over U...,Although these companies are from different se...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-23 14:00:00+00:00,"ROUND ROCK, TEXAS - APRIL 21: A Union Pacific ...",ROUND ROCK TEXAS APRIL A Union Pacific freight...,ROUND ROCK TEXAS APRIL Union Pacific freight t...
97,"{'id': None, 'name': 'Forbes'}","Trefis Team, Contributor, \n Trefis Team, Cont...","Up 130% In The Last 12 Months, Will AI Power A...",We project that earnings will come in at about...,https://www.forbes.com/sites/greatspeculations...,https://imageio.forbes.com/specials-images/ima...,2024-01-22 12:00:00+00:00,"DANA POINT, CALIFORNIA - SEPTEMBER 26: Dr. Lis...",DANA POINT CALIFORNIA SEPTEMBER Dr Lisa Su Cha...,DANA POINT CALIFORNIA SEPTEMBER Dr Lisa Su Cha...
98,"{'id': None, 'name': 'CleanTechnica'}",Carolyn Fortuna,Tesla’s Recent Q4 & FY 2023 Update Makes Stock...,"If you’re a small investor in Tesla like I am,...",https://cleantechnica.com/2024/01/29/teslas-re...,https://cleantechnica.com/wp-content/uploads/2...,2024-01-29 12:18:58+00:00,Sign up for daily news updates from CleanTechn...,Sign up for daily news update

In [10]:
df.to_csv('articles_cleaned.csv', index=False)

In [9]:
print(df.dtypes)

source                       object
author                       object
title                        object
description                  object
url                          object
urlToImage                   object
publishedAt     datetime64[ns, UTC]
content                      object
cleaned text                 object
no_sw_text                   object
dtype: object


In [14]:
df1=pd.read_csv('tsla_merged1.csv')
print(df1.dtypes)

submission_id       object
submission_title    object
date                object
comment_text        object
upvotes              int64
dtype: object
